<a href="https://colab.research.google.com/github/Harischandra359/hds5210-2023/blob/main/midterm/midterm-2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [85]:
def calculate_priest_prob(sex, age, resp_rate, oxy_saturation, heart_rate, systolic_bp, temp, alert, oxy_src, performance):
    """
    This functions calculate priest Clinical Severity Score and return the risk probability.

    Parameters:
    sex (str): The patient's sex.
    age (int): The patient's age.
    resp_rate (int): Respiratory rate.
    oxy_saturation (float): Oxygen saturation level.
    heart_rate (int): Heart rate.
    systolic_bp (int): Systolic blood pressure.
    temp (float): Body temperature.
    alert (str): Alertness status.
    oxygen_src (str): Source of inspired oxygen.
    performance (str): Performance status.

    The function returns float value
    """

    sex = sex.lower()
    alert = alert.lower()
    oxy_src = oxy_src.lower()
    performance = performance.lower()

    sex_score = age_score = resp_rate_score = oxy_sat_score = heart_rate_score = bp_score = temp_score = alert_score = oxy_src_score = performance_score = 0

    if sex == 'male':
        sex_score = 1
    elif sex == 'female':
        sex_score = 0
    else:
        return None

    if 16 <= age <= 49:
        age_score = 0
    elif 50 <= age <= 65:
        age_score = 2
    elif 66 <= age <= 80:
        age_score = 3
    elif age > 80:
        age_score = 4
    else:
        return None

    if resp_rate < 9 or resp_rate > 24:
        resp_rate_score = 3
    elif 9 <= resp_rate <= 11:
        resp_rate_score = 1
    elif 21 <= resp_rate <= 24:
        resp_rate_score = 2
    elif 12 <= resp_rate <= 20:
        resp_rate_score = 0
    else:
        return None

    if oxy_saturation >= 0.95:
        oxy_sat_score = 0
    elif oxy_saturation >= 0.94:
        oxy_sat_score = 1
    elif oxy_saturation >= 0.92:
        oxy_sat_score = 2
    elif oxy_saturation < 0.92:
        oxy_sat_score = 3
    else:
        return None

    if 51 <= heart_rate <= 90:
        heart_rate_score = 0
    elif (41 <= heart_rate <= 50) or (91 <= heart_rate <= 110):
        heart_rate_score = 1
    elif heart_rate < 41 or heart_rate > 130:
        heart_rate_score = 3
    elif 111 <= heart_rate <= 130:
        heart_rate_score = 2
    else:
        return None

    if 111 <= systolic_bp <= 219:
        bp_score = 0
    elif 101 <= systolic_bp <= 110:
        bp_score = 1
    elif 91 <= systolic_bp <= 100:
        bp_score = 2
    elif systolic_bp < 91 or systolic_bp > 219:
        bp_score = 3
    else:
        return None

    if 36.1 <= temp <= 38:
        temp_score = 0
    elif (35.1 <= temp <= 36) or (38.1 <= temp <= 39):
        temp_score = 1
    elif temp > 39:
        temp_score = 2
    elif temp < 35.1:
        temp_score = 3
    else:
        return None

    if alert == 'alert':
        alert_score = 0
    elif alert == 'confused or not alert':
        alert_score = 3
    else:
        return None

    if oxy_src == 'air':
        oxy_src_score = 0
    elif oxy_src == 'supplemental oxygen':
        oxy_src_score = 2
    else:
        return None

    if performance == 'unrestricted normal activity':
        performance_score = 0
    elif performance == 'limited strenuous activity, can do light activity':
        performance_score = 1
    elif performance == 'limited activity, can self-care':
        performance_score = 2
    elif performance == 'limited self-care':
        performance_score = 3
    elif performance == 'bed/chair bound, no self-care':
        performance_score = 4
    else:
        return None

    priest_score = sex_score + age_score + resp_rate_score + oxy_sat_score + heart_rate_score + bp_score + temp_score + alert_score + oxy_src_score + performance_score

    if 17 <= priest_score <= 25:
        priest_probability = 0.59
    elif priest_score >= 26:
        priest_probability = 0.99
    else:
        priest_probability = 0.0

    return priest_probability

# Test cases
print(calculate_priest_prob('male', 55, 18, 0.95, 85, 115, 36.9, 'confused or not alert', 'supplemental oxygen', 'limited self-care'))
print(calculate_priest_prob('female', 70, 23, 0.91, 100, 140, 38.2, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care'))
print(calculate_priest_prob('unknown', 45, 20, 0.98, 75, 125, 37.3, 'Alert', 'Air', 'Unrestricted normal activity'))



0.0
0.59
None


In [86]:
import doctest
doctest.run_docstring_examples(calculate_priest_prob, globals(),verbose=True)

Finding tests in NoName


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [87]:
import requests

def find_hospital(age, sex, risk):
    """
    (int, str, float) -> str

    Find a hospital based on age, sex, and risk percentage using a REST web service.

    >> find_hospital(40, 'male', 0.1)
    Southwest Hospital and Medical Center

    >> find_hospital(35, 'female', 0.05)
    Select Specialty Hospital - Northeast Atlanta

    >> find_hospital(50, 'male', 0.8)
    Emory Dunwoody Medical Center

    """
    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    params = {
        'age': age,
        'sex': sex,
        'risk_pct': risk
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            hospital_name = response.json().get('hospital')
            return hospital_name
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

In [88]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [89]:
import requests

def get_hospital_address(hospital_name):
    """
    (str) -> str

    Get the address of a hospital based on its name from a JSON file.

    >> get_address("COOK MEDICAL CENTER"
    706 NORTH PARRISH AVENUE

    >> get_address("INVALID HOSPITAL")
    None

    >> get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL - NORTH CAMPUS")
    2000 PALMYRA ROAD
    """
    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            hospital_data = response.json()
            if hospital_name in hospital_data:
                return hospital_data[hospital_name]["ADDRESS"]
            else:
                return None
        else:
            return None

    except Exception as e:
        print(f"Exception message: {e}")
        return None

# Test cases
print(get_hospital_address("COOK MEDICAL CENTER"))  # Test case with a valid hospital name
print(get_hospital_address("INVALID HOSPITAL"))  # Test case with an invalid hospital name
print(get_hospital_address("PHOEBE PUTNEY MEMORIAL HOSPITAL - NORTH CAMPUS"))  # Another valid test case


706 NORTH PARRISH AVENUE
None
2000 PALMYRA ROAD


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [90]:
import requests

def process_patients(file_path):
    """
    Process patient data from a pipe-delimited file, calculate PRIEST scores, determine hospital_names, and get addresses.

    Args:
        file_location (str): URL of the pipe-delimited file containing patient data.

    Returns:
        dict: Dictionary with patient numbers as keys and their information, hospital_name, and address as values.
    """
    response = requests.get(file_path)

    if response.status_code == 200:
        data = response.text.split('\n')
        processed_data = {}
        for id, row in enumerate(data):
            # Skip the header row
            if id == 0:
                continue

            columns = row.strip().split('|')

            if len(columns) != 11:
                print(f"Skipping row {id + 1} due to incorrect data format.")
                continue

            patient_number, sex, age, breath, oxygen_sat, heart_rate, systolic_bp, temp, alertness, inspired, status = columns

            # Convert numerical columns to appropriate data types
            age = int(age)
            breath = int(breath)
            oxygen_sat = float(oxygen_sat)
            heart_rate = int(heart_rate)
            systolic_bp = int(systolic_bp)
            temp = float(temp)

            risk = calculate_priest_prob(sex, age, breath, oxygen_sat, heart_rate, systolic_bp, temp, alertness, inspired, status)

            hospital_name = find_hospital_name(age, sex.lower(), risk)
            address = get_hospital_address(hospital_name) if hospital_name else None

            patient_info = {
                "sex": sex,
                "age": age,
                "breath": breath,
                "oxygen_sat": oxygen_sat,
                "heart_rate": heart_rate,
                "systolic_bp": systolic_bp,
                "temp": temp,
                "alertness": alertness,
                "inspired": inspired,
                "status": status,
                "hospital_name": hospital_name,
                "address": address
            }
            processed_data[patient_number] = patient_info

        return processed_data
    else:
        return None

file_path = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
processed_data = process_patients(file_path)
print(processed_data)

Skipping row 101 due to incorrect data format.
{'E9559': {'sex': 'FEMALE', 'age': 40, 'breath': 24, 'oxygen_sat': 0.96, 'heart_rate': 105, 'systolic_bp': 115, 'temp': 34.9, 'alertness': 'ALERT', 'inspired': 'AIR', 'status': 'unrestricted normal activity', 'hospital_name': None, 'address': None}, 'E9385': {'sex': 'Female', 'age': 51, 'breath': 19, 'oxygen_sat': 0.99, 'heart_rate': 90, 'systolic_bp': 91, 'temp': 38.1, 'alertness': 'ALERT', 'inspired': 'supplemental oxygen', 'status': 'unrestricted normal activity', 'hospital_name': None, 'address': None}, 'E3067': {'sex': 'female', 'age': 40, 'breath': 29, 'oxygen_sat': 0.96, 'heart_rate': 105, 'systolic_bp': 95, 'temp': 38.1, 'alertness': 'ALERT', 'inspired': 'air', 'status': 'unrestricted normal activity', 'hospital_name': None, 'address': None}, 'E9422': {'sex': 'FEMALE', 'age': 66, 'breath': 19, 'oxygen_sat': 0.96, 'heart_rate': 135, 'systolic_bp': 115, 'temp': 38.1, 'alertness': 'ALERT', 'inspired': 'AIR', 'status': 'bed/chair bound

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [91]:
file_path = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"

url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

response = requests.get(url)
if response.status_code == 200:
    results = response.json()
else:
    print("Failed to download expected results.")
    results = {}

processed_patients_data = process_patients(file_path)

for patient_number, expected_info in results.items():
    if patient_number in processed_patients_data:
        processed_info = processed_patients_data[patient_number]
        if (
            expected_info[0].lower() == processed_info["sex"].lower()
            and expected_info[1] == processed_info["age"]
            and expected_info[2] == processed_info["breath"]
            and expected_info[3] == processed_info["oxygen_sat"]
            and expected_info[4] == processed_info["heart_rate"]
            and expected_info[5] == processed_info["systolic_bp"]
            and expected_info[6] == processed_info["temp"]
            and expected_info[7].lower() == processed_info["alertness"].lower()
            and expected_info[8].lower() == processed_info["inspired"].lower()
            and expected_info[9].lower() == processed_info["status"].lower()
            and (processed_info["hospital_name"] is None or expected_info[11].lower() == processed_info["hospital"].lower())
            and (processed_info["address"] is None or expected_info[12].lower() == processed_info["address"].lower())
        ):
            print(f"Patient {patient_number}: processed data matched the expected results.")
        else:
            print(f"Patient {patient_number}: processed data not matched the expected results.")
    else:
       print(f"Patient {patient_number} not found in processed data.")

Skipping row 101 due to incorrect data format.
Patient E9559: processed data matched the expected results.
Patient E9385: processed data matched the expected results.
Patient E3067: processed data matched the expected results.
Patient E9422: processed data matched the expected results.
Patient E8661: processed data matched the expected results.
Patient E6235: processed data matched the expected results.
Patient E4451: processed data matched the expected results.
Patient E8433: processed data matched the expected results.
Patient E7593: processed data matched the expected results.
Patient E3296: processed data matched the expected results.
Patient E4157: processed data matched the expected results.
Patient E7702: processed data matched the expected results.
Patient E8158: processed data matched the expected results.
Patient E4795: processed data matched the expected results.
Patient E8800: processed data matched the expected results.
Patient E4855: processed data matched the expected re

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

In [92]:
def summarize_results(patients_data):
    """
    Summarizes patient results based on a new business rule and counts how many patients are being assigned to different locations.

    Parameters:
        patients_data (dict): Dictionary containing patient information, hospitals, and addresses.

    Returns a dictionary of which represents the summary of results
    """
    result_summary = {"Home": 0}

    for patient in patients_data.values():
        patient_hospital = patient.get("hospital")
        patient_risk = patient.get("risk")
        if patient_risk and patient_risk < 0.10:
            patient_hospital = "Home"
        if patient_hospital not in result_summary:
            result_summary[patient_hospital] = 1
        else:
            result_summary[patient_hospital] += 1
    return result_summary


In [93]:
file_location = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
patients = process_patients(file_location)
summary = summarize_results(patients)
print(summary)

Skipping row 101 due to incorrect data format.
{'Home': 0, None: 99}


---